In [1]:
import numpy as np
import pandas as pd
import json
import csv

from bokeh.plotting import figure
from bokeh.io import output_file, show, save
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.core.properties import value
from bokeh.models import PanTool, ResetTool, WheelZoomTool, HoverTool, LassoSelectTool, BoxSelectTool


In [2]:
# Laad de dataset en de participant set
df = pd.read_json("dataset.json", lines=True)
df_p = pd.read_json('dataset_incidents.json', lines=True)

In [10]:
# maakt nieuwe columns met de maand en het jaar

df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# zet de index op month
df_maand = df.set_index(['month'])


# zet index op jaar en dan maand
df_tijd = df.set_index(['year','month'])



In [11]:
# selecteer op man en maak onderscheid suspect en victim
man = df_p.loc[df_p['participant_gender']=='Male']
suspect_man = man.loc[man['participant_type']=='Subject-Suspect']
victim_man = man.loc[man['participant_type']=='Victim']

# maak een dict met de het aantal gevallen per leeftijd voor suspect en victim
leeftijden_man_suspect = suspect_man['participant_age'].value_counts().to_dict()
leeftijden_man_victim = victim_man['participant_age'].value_counts().to_dict()

# selecteer op vrouw en maak onderscheid suspect en victim
woman = df_p.loc[df_p['participant_gender']=='Female']
suspect_woman = woman.loc[woman['participant_type']=='Subject-Suspect']
victim_woman = woman.loc[woman['participant_type']=='Victim']

# maak een dict met de het aantal gevallen per leeftijd voor suspect 
leeftijden_woman_suspect = suspect_woman['participant_age'].value_counts().to_dict()
leeftijden_woman_victim = victim_woman['participant_age'].value_counts().to_dict()



# order de dicts en zet in lijst, nieuwe variabelen voor elke groep
x = []
y = []
for  k, v in sorted(leeftijden_man_victim.items()):
   
    x.append(k)
    y.append(v)

x1 = []
y1 = []
for  k, v in sorted(leeftijden_man_suspect.items()):
   
    x1.append(k)
    y1.append(v)

x2 = []
y2 = []
for  k, v in sorted(leeftijden_woman_victim.items()):
   
    x2.append(k)
    y2.append(v)
    
x3 = []
y3 = []
for  k, v in sorted(leeftijden_woman_suspect.items()):
   
    x3.append(k)
    y3.append(v)


In [12]:
# Maak plots


output_file("Gender_age_division_type.html")

p = figure(x_range = (0,100), y_range=(0,6000), title = 'Age division by gender')

p.line(x, y, line_width=2, line_color = "#47A8E5")
p.circle(x, y, size=2,  fill_color = '#47A8E5', line_color="#47A8E5", line_width=3, legend='Male Victim')

p.line(x1, y1, line_width=2, line_color = '#133A7C')
p.circle(x1, y1, size=2,  fill_color = '#133A7C', line_color="#133A7C", line_width=3, legend='Male Suspect')

p.line(x2, y2, line_width=2, line_color = "#FF0800")
p.circle(x2, y2, size=2,  fill_color = '#FF0800', line_color="#FF0800", line_width=3, legend='Female Victim')

p.line(x3, y3, line_width=2, line_color = "#800000")
p.circle(x3, y3, size=2,  fill_color = '#800000', line_color="#800000", line_width=3, legend='Female Suspect')



p.xaxis.axis_label = "Age"
p.xaxis.axis_label_text_color = "#aa6666"
p.xaxis.axis_label_standoff = 30

p.yaxis.axis_label = "Number of Incidents"
p.yaxis.axis_label_text_color = '#aa6666'

show(p)




In [37]:
#maak een df met de jaren en maand als index
df_maandcount = lekker.groupby(['year','month']).size().reset_index(name='count').to_dict()

#voeg bijbehorende key en values toe aan lijst 
maanden = []
count = []
for i in range(1,63):
    maanden.append(i+1)
    count.append(df_maandcount['count'][i])
    
# maak een lineare regressie lijst
regression = np.polyfit(maanden[12:], count[12:], 1)

print(regression)

# nu de waarden invullen voor de regressie
r_x, r_y = zip(*((i, i*regression[0] + regression[1]) for i in range(12,64)))


[  20.61056423 3906.55327731]


In [38]:
# Plot met lineare regressie vanaf 2014

output_file("Month_incidents_troughout.html")

p = figure(x_range = (1,63), y_range=(0,5800), title = 'Number of incidents per month')

p.line(maanden, count, line_width=2, line_color = "#47A8E5")
p.circle(maanden, count, size=2,  fill_color = '#47A8E5', line_color="#47A8E5", line_width=3)


p.line(r_x, r_y, line_width=2, line_color = "red")


p.xaxis.axis_label = "Month"
p.xaxis.axis_label_text_color = "#aa6666"
p.xaxis.axis_label_standoff = 30

p.yaxis.axis_label = "Number of Incidents"
p.yaxis.axis_label_text_color = '#aa6666'

show(p)
